<a href="https://colab.research.google.com/github/CK705/Test1/blob/master/NIFTY_50_Stock_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score  # Import MSE and R-squared

def predict_stock_price(ticker, start_date, end_date, prediction_days=60):
    """
    Predicts stock prices using an LSTM model with a 70/30 training/testing split.

    Args:
        ticker (str): Stock ticker symbol (e.g., "RELIANCE.NS").
        start_date (str): Start date for data retrieval (e.g., "2010-01-01").
        end_date (str): End date for data retrieval (e.g., "2025-01-01").
        prediction_days (int): Number of days to use for prediction.

    Returns:
        tuple: Predicted prices and actual prices for plotting.
    """
    try:
        # 1. Download Stock Data
        data = yf.download(ticker, start=start_date, end=end_date)
        df = data['Close'].values.reshape(-1, 1)

        # 2. Scale the Data
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(df)

        # 3. Split Data into Training and Testing Sets (70/30 Split)
        train_size = int(len(scaled_data) * 0.7)
        train_data = scaled_data[:train_size]
        test_data = scaled_data[train_size - prediction_days:]  # Include overlap for test data sequences

        # 4. Prepare the Training Data
        x_train, y_train = [], []
        for i in range(prediction_days, len(train_data)):
            x_train.append(train_data[i - prediction_days:i, 0])
            y_train.append(train_data[i, 0])
        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

        # 5. Prepare the Test Data
        x_test = []
        for i in range(prediction_days, len(test_data)):
            x_test.append(test_data[i - prediction_days:i, 0])
        x_test = np.array(x_test)
        x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
        actual_prices = data['Close'].values[train_size:]  # Get actual prices for the test period

        # 6. Build the LSTM Model
        model = Sequential()
        model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
        model.add(LSTM(units=50))
        model.add(Dense(units=1))

        # 7. Train the Model
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, epochs=25, batch_size=32)

        # 8. Make Predictions
        predicted_prices = model.predict(x_test)
        predicted_prices = scaler.inverse_transform(predicted_prices)

        # 9. Return Results
        return actual_prices, predicted_prices

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None


# 10. Example Usage
ticker = "^NSEI"  # Correct ticker for Nifty 50
start_date = '2010-01-01'
end_date = '2025-01-01'

actual_prices, predicted_prices = predict_stock_price(ticker, start_date, end_date)

# 11. Plot the Results
if actual_prices is not None and predicted_prices is not None:
    plt.plot(actual_prices, color='blue', label=f'Actual {ticker} Price')
    plt.plot(predicted_prices, color='green', label=f'Predicted {ticker} Price')
    plt.title(f'{ticker} Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel(f'{ticker} Stock Price')
    plt.legend()
    plt.show()

    # 12. Calculate and Print MSE and R-squared
    mse = mean_squared_error(actual_prices, predicted_prices)
    r_squared = r2_score(actual_prices, predicted_prices)
    print(f'Mean Squared Error: {mse}')
    print(f'R-squared: {r_squared}')

[*********************100%***********************]  1 of 1 completed

Epoch 1/25



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 